In [1]:
import sys
from utils import *
sys.modules['pkg_resources'] = None

/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


getting filepaths from cosmosdb
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln


In [2]:
print("set environment variables")
os.environ['OPENAI_API_KEY'] = get_secret("chatKeys", "openaiKey")   
os.environ['storage_account_name'] = 'chatgptv2stn'
os.environ['container_name'] = 'chatgpt-ctn'
os.environ['resource_group_name'] ='chatgptGp'
os.environ['cosmosdb_acc'] ='chatgptdb-acn'
os.environ['database_name']='chatgptdb-dbn'
os.environ['collection_name']='chatgptdb-cln'        
pinecone_dict = get_pinecone_keys()
pinecone_jar, cosmos_jar = set_spark_liraries()    

set environment variables


In [3]:
spark = SparkSession.builder\
        .appName("chatgpt")\
        .config("spark.jars.packages", f"{pinecone_jar}")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

23/03/26 19:29:39 WARN Utils: Your hostname, codespaces-670d97 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/03/26 19:29:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.pinecone#spark-pinecone_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f5aaf05f-46cc-453f-8af6-7914037bd443;1.0
	confs: [default]
	found io.pinecone#spark-pinecone_2.13;0.1.1 in central
	found io.pinecone#pinecone-client;0.2.2 in central
	found io.netty#netty-tcnative-boringssl-static;2.0.50.Final in central
	found io.netty#netty-tcnative-classes;2.0.50.Final in central
	found io.grpc#grpc-protobuf;1.47.0 in central
	found io.grpc#grpc-api;1.47.0 in central
	found io.grpc#grpc-context;1.47.0 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
	found com.google.protobuf#protobuf-java;3.19.2 in central
	found com.google.api.grpc#proto-google-common-protos;2.0.1 in central
	found io.grpc#grpc-protobuf-lite;1.47.0 in central
	found io.grpc#g

23/03/26 19:29:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
preprocess_text_list =  spark.sparkContext.parallelize(extract_text_from_container()).collect()


get environment variables
Failed to extract text from PDF using Tika or pdfplumber
Failed to extract text from PDF using Tika or pdfplumber
Failed to extract text from PDF using Tika or pdfplumber
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 50645 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 53836 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 43515 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 40730 characters from PDF using Tika
Successfully extracted 75278 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 61900 characters from PDF using Tika
Successfully extracted 32119 characters from PDF using Tika
Failed to extract text from PDF using Tika or pdfplumber
Successfully extracted 71598 characters f

In [16]:
spark.sparkContext.parallelize(preprocess_text_list[0]).map(lambda x: {
                                            "Filepath": x[0],
                                            "Metadata": {
                                                "folder": extract_title(x[0])[0],
                                                "typeofDoc": extract_title(x[0])[1],
                                                "subject": extract_title(x[0])[2],
                                                "author": extract_title(x[0])[3],
                                                "title": extract_title(x[0])[4].split('.')[0]
                                            },
                                            "text": x[1],
                                            "summary": cheaper_summarizer(x[1], extract_title(x[0])),
                                            "id": create_id(
                                                extract_title(x[0])[0],  # folder
                                                extract_title(x[0])[1],  # typeofDoc
                                                extract_title(x[0])[2],  # subject
                                                extract_title(x[0])[3],  # author
                                                extract_title(x[0])[4].split('.')[0],  # title
                                            ),
                                            "uploadDate": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                        }).foreachPartition(lambda x: write_to_cosmosdb(x))

getting filepaths from cosmosdb                                     (0 + 4) / 4]
getting filepaths from cosmosdb
getting filepaths from cosmosdb
getting filepaths from cosmosdb
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln


23/03/26 19:44:12 ERROR Executor: Exception in task 3.0 in stage 5.0 (TID 23)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 5.0 failed 1 times, most recent failure: Lost task 3.0 in stage 5.0 (TID 23) (92bf1a1f-7cfc-4d0e-8a98-914fd2ab575b.internal.cloudapp.net executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", lin

In [12]:
load_to_cosmosdb_rdd = spark.sparkContext.parallelize(preprocess_text_list[0]).map(lambda x: (x[0], preprocess_text(x[1]))).\
                                    map(lambda x: {
                                            "Filepath": x[0],
                                            "Metadata": {
                                                "folder": extract_title(x[0])[0],
                                                "typeofDoc": extract_title(x[0])[1],
                                                "subject": extract_title(x[0])[2],
                                                "author": extract_title(x[0])[3],
                                                "title": extract_title(x[0])[4].split('.')[0]
                                            },
                                            "text": x[1],
                                            "summary": cheaper_summarizer(x[1], extract_title(x[0])),
                                            "id": create_id(
                                                extract_title(x[0])[0],  # folder
                                                extract_title(x[0])[1],  # typeofDoc
                                                extract_title(x[0])[2],  # subject
                                                extract_title(x[0])[3],  # author
                                                extract_title(x[0])[4].split('.')[0],  # title
                                            ),
                                            "uploadDate": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                        }).foreachPartition(lambda x: write_to_cosmosdb(x))

getting filepaths from cosmosdb                                     (0 + 4) / 4]
getting filepaths from cosmosdb
getting filepaths from cosmosdb
getting filepaths from cosmosdb
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln
chatgptGp chatgptdb-acn chatgptdb-dbn chatgptdb-cln


23/03/26 19:40:22 ERROR Executor: Exception in task 1.0 in stage 3.0 (TID 13)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3.0 (TID 13) (92bf1a1f-7cfc-4d0e-8a98-914fd2ab575b.internal.cloudapp.net executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/codespace/.python/current/lib/python3.10/site-packages/pyspark/rdd.py", lin

In [ ]:




def main():

    #set variables to be used in this ETL process

    print("set environment variables")
    os.environ['OPENAI_API_KEY'] = get_secret("chatKeys", "openaiKey")   
    os.environ['storage_account_name'] = 'chatgptv2stn'
    os.environ['container_name'] = 'chatgpt-ctn'
    os.environ['resource_group_name'] ='chatgptGp'
    os.environ['cosmosdb_acc'] ='chatgptdb-acn'
    os.environ['database_name']='chatgptdb-dbn'
    os.environ['collection_name']='chatgptdb-cln'        
    pinecone_dict = get_pinecone_keys()
    pinecone_jar, cosmos_jar = set_spark_liraries()    
    
    
    

        
    #blob storage loading

    print("load blob storage")
    pdf_paths = [item for item in list_pdfblobs()][:1]
    print(f"number of pdf files: {len(pdf_paths)}")

    
    print("starting spark session")    
    spark = SparkSession.builder\
        .appName("chatgpt")\
        .config("spark.jars.packages", f"{pinecone_jar}")\
        .getOrCreate()

    spark.sparkContext.setLogLevel("ERROR")

    print(pdf_paths)        
    
    print("create the cosmosdb rdd")

    preprocess_text_list =  spark.sparkContext.parallelize(extract_text_from_container()).collect()

    
    load_to_cosmosdb_rdd = spark.sparkContext.parallelize(preprocess_text_list).map(lambda x: (x[0], preprocess_text(x[1]))).\
                                    map(lambda x: {
                                            "Filepath": x[0],
                                            "Metadata": {
                                                "folder": extract_title(x[0])[0],
                                                "typeofDoc": extract_title(x[0])[1],
                                                "subject": extract_title(x[0])[2],
                                                "author": extract_title(x[0])[3],
                                                "title": extract_title(x[0])[4].split('.')[0]
                                            },
                                            "text": x[1],
                                            "summary": cheaper_summarizer(x[1], extract_title(x[0])),
                                            "id": create_id(
                                                extract_title(x[0])[0],  # folder
                                                extract_title(x[0])[1],  # typeofDoc
                                                extract_title(x[0])[2],  # subject
                                                extract_title(x[0])[3],  # author
                                                extract_title(x[0])[4].split('.')[0],  # title
                                            ),
                                            "uploadDate": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                        }).foreachPartition(lambda x: write_to_cosmosdb(x))
    

    # print("create the pinecone rdd and write to pinecone")
    # load_data_to_pinecone_rdff = spark.sparkContext.parallelize(get_data_from_cosmosdb()).map(lambda x : ([x['text'],x['Metadata']])).\
    #                                             map(lambda x : get_pincone_pdfdata(x[0],x[1])).\
    #                                             foreach(lambda x : upsert_pinecone_data(x))
                                                

    
    
    
if __name__ == "__main__":
    main()

    
